In [0]:
from google.colab import drive

In [0]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
data_path = '/content/gdrive/My Drive/grid_models/'

In [0]:
!unzip -q '/content/gdrive/My Drive/grid_models/FlipkartGridStage2DataSetImages.zip'

In [0]:
import pandas as pd
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [0]:
from keras.layers import Conv2D, Activation, MaxPooling2D, Flatten, Dense, Input, UpSampling2D
from keras.layers import BatchNormalization, LeakyReLU, add, GlobalAveragePooling2D, Reshape
from keras.models import Model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import Sequence
from keras.regularizers import l2
import keras.backend as K
import keras

Using TensorFlow backend.


In [0]:
df = pd.read_csv(data_path + 'training_set.csv')
filenames = df['image_name'].tolist()
x1 = df['x1'].tolist()
x2 = df['x2'].tolist()
y1 = df['y1'].tolist()
y2 = df['y2'].tolist()
labels = []
for i in range(len(filenames)):
    temp = []
    temp.append(x1[i])
    temp.append(x2[i])
    temp.append(y1[i])
    temp.append(y2[i])
    labels.append(temp)
labels = np.array(labels)
filenames = np.array(filenames)

In [0]:
p = np.random.permutation([i for i in range(len(filenames))])
filenames_train = np.array([filenames[i] for i in p[:22000]])
filenames_validation = np.array([filenames[i] for i in p[22000:]])
labels_train = np.array([labels[i] for i in p[:22000]])
labels_validation = np.array([labels[i] for i in p[22000:]])

In [0]:
import keras
from multiprocessing import Pool
def return_bb(output, threshold):
    wh = np.where(output >= threshold)
    if wh[0].shape[0] == 0:
        y_min = 100
        y_max = 380
        x_min = 150
        x_max = 500
    else:
      y_min = np.min(wh[0])
      y_max = np.max(wh[0])
      x_min = np.min(wh[1])
      x_max = np.max(wh[1])
    temp = []
    temp.append(x_min)
    temp.append(x_max)
    temp.append(y_min)
    temp.append(y_max)
    return temp
    
def return_img(filename):
    img = cv2.imread('images/' + filename)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img / 255.
    return img
    


class validation_score(keras.callbacks.Callback):
    def on_epoch_end(self, batch, logs={}):
        boxes_5 = []
        boxes_7 = []
        boxes_9 = []
        boxes_95 = []
        for i in range(int(len(filenames_validation)/8)):
            temp_filenames = []
            for j in range(i*8, (i+1)*8):
                temp_filenames.append(filenames_validation[j])
            p = Pool()
            img_batch = p.map(return_img, temp_filenames)
            img_batch = np.array(img_batch)
            #print ('Batch_Done')
            out = model.predict(img_batch)
            #print ('Prediction Done')
            p = Pool()
            #temp = p.map(return_bb, out, 0.5)
            #boxes_5 = boxes_5 + temp
            temp = p.map(return_bb, out, 0.7)
            boxes_7 = boxes_7 + temp
            #temp = p.map(return_bb, out, 0.9)
            #boxes_9 = boxes_9 + temp
            #temp = p.map(return_bb, out, 0.95)
            #boxes_95 = boxes_95 + temp
            #print ('0.5 score: {}'.format(iou_score(labels_validation, boxes_5)))
            print ('0.7 score: {}'.format(iou_score(labels_validation, boxes_7)))
            #print ('0.9 score: {}'.format(iou_score(labels_validation, boxes_9)))
            #print ('0.95 score: {}'.format(iou_score(labels_validation, boxes_95)))
def iou_score(labels, boxes):
    mean_iou = 0.0
    for i in range(len(labels)):
        x1_a = labels[i][0]
        x2_a = labels[i][1]
        y1_a = labels[i][2]
        y2_a = labels[i][3]
        x1_b = boxes[i][0]
        x2_b = boxes[i][1]
        y1_b = boxes[i][2]
        y2_b = boxes[i][3]
        area1 = (x2_a - x1_a) * (y2_a - y1_a)
        area2 = (x2_b - x1_b) * (y2_b - y1_b)
        intersection = (min(x2_a, x2_b) - max(x1_a, x1_b)) * (min(y2_a, y2_b) - max(y1_a, y1_b))
        iou = float(intersection) / float(area1 + area2 - intersection)
        mean_iou += iou
    mean_iou = float(mean_iou) / float(len(labels))
    return mean_iou

In [0]:
def model_temp(input_shape):
    X_input = Input(input_shape)
    X = Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu')(X_input)
    X = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same')(X)
    X = Conv2D(filters=128, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu')(X)
    X = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same')(X)
    X = Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu')(X)
    X = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same')(X)    
    X = Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu')(X)
    X = UpSampling2D(size=(2, 2))(X)
    X = Conv2D(filters=128, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu')(X)
    X = UpSampling2D(size=(2, 2))(X)
    X = Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu')(X)
    X = UpSampling2D(size=(2, 2))(X)
    X = Conv2D(filters=1, kernel_size=(1, 1), strides=(1, 1), padding='same', activation='sigmoid')(X)
    X = Reshape([480, 640])(X)
    return Model(X_input, X)
def auto_encoder(input_shape):
    X_input = Input(input_shape)
    X = Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu')(X_input)
    X = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same')(X)
    X = Conv2D(filters=128, kernel_size=(5, 5), strides=(1, 1), padding='same', activation='relu')(X)
    X = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same')(X)
    X = Conv2D(filters=256, kernel_size=(5, 5), strides=(1, 1), padding='same', activation='relu')(X)
    X = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same')(X)
    X = Conv2D(filters=512, kernel_size=(7, 7), strides=(1, 1), padding='same', activation='relu')(X)
    X = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same')(X)
    X = Conv2D(filters=512, kernel_size=(7, 7), strides=(1, 1), padding='same', activation='relu')(X)
    X = UpSampling2D(size=(2, 2))(X)
    X = Conv2D(filters=256, kernel_size=(5, 5), strides=(1, 1), padding='same', activation='relu')(X)
    X = UpSampling2D(size=(2, 2))(X)
    X = Conv2D(filters=128, kernel_size=(5, 5), strides=(1, 1), padding='same', activation='relu')(X)
    X = UpSampling2D(size=(2, 2))(X)
    X = Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu')(X)
    X = UpSampling2D(size=(2, 2))(X)
    X = Conv2D(filters=1, kernel_size=(1, 1), strides=(1, 1), padding='same', activation='sigmoid')(X)
    X = Reshape([480, 640])(X)
    return Model(X_input, X)
model = auto_encoder((480, 640, 3))

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 480, 640, 3)       0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 480, 640, 64)      1792      
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 240, 320, 64)      0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 240, 320, 128)     204928    
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 120, 160, 128)     0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 120, 160, 256)     819456    
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 60, 80, 256)       0         
__________

In [0]:
class DataGenerator(Sequence):     
    def __init__(self, image_filenames, labels, batch_size, shuffle=True):
        self.image_filenames, self.labels = image_filenames, labels
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.image_filenames) / self.batch_size))
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.image_filenames))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __getitem__(self, idx):
        indexes = self.indexes[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_x = [self.image_filenames[i] for i in indexes]
        batch_y = [self.labels[i] for i in indexes]
        batch_x, batch_y = self.generate_data(batch_x, batch_y)
        return batch_x, batch_y 

    def generate_data(self, batch_x, batch_y):
        x = []
        y = []
        for i in batch_x:
            img = cv2.imread('images/' + i)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = img / 255.
            x.append(img)
        for i in batch_y:
            x_min = i[0]
            x_max = i[1]
            y_min = i[2]
            y_max = i[3]
            temp = np.zeros((480, 640))
            temp[y_min:y_max, x_min:x_max] = 1.
            y.append(temp)
        return np.array(x), np.array(y)

In [0]:
from keras.optimizers import SGD, RMSprop, Adam
from keras.models import load_model

In [0]:
checkpoint = ModelCheckpoint(data_path + "m5.{epoch:02d}-{loss:.2f}.hdf5", monitor='loss', save_weights_only=False, save_best_only=True)
early_stopping = EarlyStopping(monitor='loss', min_delta=0, patience=10, verbose=1, mode='auto')
#val_score = validation_score()

In [0]:
opt = Adam(lr=0.00005)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['acc'])

In [0]:
model = load_model(data_path + 'm5.01-0.12.hdf5')

In [0]:
training_generator = DataGenerator(filenames, labels, 8)

In [0]:
model.fit_generator(generator=training_generator, use_multiprocessing=True, workers=8, epochs=20, callbacks=[checkpoint, early_stopping])

In [0]:
test_filenames = pd.read_csv(data_path + 'test_set.csv')['image_name'].tolist()

In [0]:
class testDataGenerator(Sequence):     
    def __init__(self, image_filenames, batch_size, shuffle=False):
        self.image_filenames = image_filenames
        self.batch_size = batch_size
        self.shuffle = shuffle
        #self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.image_filenames) / float(self.batch_size)))
    
#     def on_epoch_end(self):
#         'Updates indexes after each epoch'
#         self.image_filenames = np.arange(len(self.image_filenames))
#         if self.shuffle == True:
#             np.random.shuffle(self.indexes)

    def __getitem__(self, idx):
        batch_x = self.image_filenames[idx * self.batch_size:(idx + 1) * self.batch_size]
        x = self.generate_data(batch_x)
        return x

    def generate_data(self, batch_x):
        x = []
        for i in batch_x:
            #print (i)
            img = cv2.imread('images/' + i)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = img / 255.
            x.append(img)
        return np.array(x)

In [0]:
test_generator = testDataGenerator(test_filenames, 8)

In [0]:
from keras.models import load_model

In [0]:
model = load_model(data_path + 'm5.01-0.11.hdf5')

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 480, 640, 3)       0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 480, 640, 32)      896       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 240, 320, 32)      0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 240, 320, 64)      18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 120, 160, 64)      0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 120, 160, 128)     204928    
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 60, 80, 128)       0         
__________

In [0]:
len(test_filenames)

24045

In [0]:
import tqdm
from multiprocessing import Pool

In [0]:
def return_bb(output):
    wh = np.where(output >= 0.95)
    if wh[0].shape[0] == 0:
        y_min = 100
        y_max = 380
        x_min = 150
        x_max = 500
    else:
      y_min = np.min(wh[0])
      y_max = np.max(wh[0])
      x_min = np.min(wh[1])
      x_max = np.max(wh[1])
    temp = []
    temp.append(x_min)
    temp.append(x_max)
    temp.append(y_min)
    temp.append(y_max)
    return temp
    
def return_img(filename):
    img = cv2.imread('images/' + filename)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img / 255.
    return img
    

boxes = []
for i in tqdm.tqdm(range(int(len(test_filenames)/16) + 1)):
    filenames = []
    if i == 1502:
        for j in range(i*16, len(test_filenames)):
            filenames.append(test_filenames[j])
    else:
      for j in range(i*16, (i+1)*16):
          filenames.append(test_filenames[j])
    p = Pool()
    batch = p.map(return_img, filenames)
    batch = np.array(batch)
    print ('Batch_Done')
    out = model.predict(batch)
    print ('Prediction Done')
    p = Pool()
    temp = p.map(return_bb, out)
    boxes = boxes + temp
    
    


  0%|          | 0/1503 [00:00<?, ?it/s]

Batch_Done
Prediction Done



  0%|          | 1/1503 [00:11<4:49:26, 11.56s/it]

Batch_Done
Prediction Done



  0%|          | 2/1503 [00:14<3:44:07,  8.96s/it]

...

Batch_Done
Prediction Done



 78%|███████▊  | 1168/1503 [52:41<14:23,  2.58s/it]

Batch_Done
Prediction Done



 78%|███████▊  | 1169/1503 [52:43<14:14,  2.56s/it]

Batch_Done


In [0]:
df = pd.DataFrame(test_filenames, columns=['image_name'])

In [0]:
x1 = []
x2 = []
y1 = []
y2 = []
for i in boxes:
    x1.append(i[0])
    x2.append(i[1])
    y1.append(i[2])
    y2.append(i[3])

In [0]:
df['x1'] = x1
df['x2'] = x2
df['y1'] = y1
df['y2'] = y2

In [0]:
df.shape

(24045, 5)

In [0]:
df.to_csv(data_path + 'submit_final_95_temp.csv', index=False, index_label=False)

In [0]:
len(boxes)

NameError: ignored

In [0]:
a = [[1, 2], [2, 4]]

In [0]:
a = a + [[2, 4], [3, 6]]

In [0]:
a

[[1, 2], [2, 4], [2, 4], [3, 6]]

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 480, 640, 3)       0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 480, 640, 64)      1792      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 240, 320, 64)      0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 240, 320, 64)      36928     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 120, 160, 64)      0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 120, 160, 128)     73856     
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 120, 160, 128)     147584    
__________